In [5]:
import asyncio
import solara
import bleak
from device_model import DeviceModel

connected_device = solara.reactive(None)
device_model = solara.reactive(None)
acceleration = solara.reactive((0.0, 0.0, 0.0))
devices = solara.reactive([])
selected_device = solara.reactive(None)
status = solara.reactive("Idle")

# Data update callback
def update_data(device):
    acc_x = device.get("AccX")
    acc_y = device.get("AccY")
    acc_z = device.get("AccZ")
    acceleration.set((acc_x, acc_y, acc_z))

async def scan_devices():
    status.set("Scanning for devices...")
    try:
        found = await bleak.BleakScanner.discover(timeout=10.0)
        wt_devices = [d for d in found if d.name and "WT" in d.name]
        devices.set(wt_devices)
        status.set(f"Found {len(wt_devices)} WT devices.")
    except Exception as e:
        status.set(f"Error during scanning: {e}")

async def connect():
    status.set("Attempting to connect...")
    if selected_device.value is not None:
        try:
            dev_model = DeviceModel("MyBle5.0", selected_device.value, update_data)
            await dev_model.openDevice()
            device_model.set(dev_model)
            connected_device.set(True)
            status.set("Connected to device.")
        except Exception as e:
            status.set(f"Connection failed: {e}")
    else:
        status.set("No device selected.")

async def calibrate_acceleration():
    status.set("Sending calibration command...")
    if device_model.value is not None:
        try:
            await device_model.value.sendData([0xFF, 0xAA, 0x01, 0x01])
            status.set("Calibration command sent.")
        except Exception as e:
            status.set(f"Calibration failed: {e}")
    else:
        status.set("Device not connected.")

@solara.component
def Page():
    solara.Title("WT901BLE5.0 Accelerometer App")

    with solara.Column():
        solara.Markdown("## Bluetooth Devices")
        solara.Button("Scan Devices", on_click=lambda: asyncio.create_task(scan_devices()))

        if devices.value:
            options = {f"{d.name} ({d.address})": d for d in devices.value}
            selected_label = list(options.keys())[0] if options else None

            def on_select(label):
                selected_device.set(options[label])

            solara.Select(label="Select Device", values=list(options.keys()), on_value=on_select)

        if selected_device.value and not connected_device.value:
            async def on_connect_click():
                await connect()

            solara.Button("Connect", on_click=on_connect_click)

        if connected_device.value:
            solara.Markdown("## Acceleration (X, Y, Z)")
            solara.Text(f"{acceleration.value[0]:.2f}, {acceleration.value[1]:.2f}, {acceleration.value[2]:.2f}")
            solara.Button("Calibrate Accelerometer", on_click=lambda: asyncio.create_task(calibrate_acceleration()))

        solara.Markdown(f"### Status: {status.value}")

In [6]:
Page()

Cannot show ipywidgets in text